# Import Library

In [1]:
import tensorflow as tf

2023-04-18 12:11:26.703752: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


2023-04-18 11:44:54.382717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-18 11:44:58.612904: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-18 11:44:58.614173: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-18 11:44:58.654852: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operation

In [4]:
tf.test.is_built_with_cuda()

True

In [2]:
import os
#import cv2
import sys
import gc
import numpy as np
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Conv3D, MaxPooling2D, AveragePooling2D, Flatten, Dense, LSTM, TimeDistributed, GlobalAveragePooling2D
# from tensorflow.keras.applications.resnet50 import ResNet50
# from tensorflow.keras.utils import to_categorical
# from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
# from tensorflow.keras.utils import plot_model
# import keras.utils as image

In [4]:
gc.collect()

0

# Proprocessing

## Set Constant

In [7]:
# Define constants
DATA_DIR = 'Validate'
DATA2_DIR = 'Validate2'
CATEGORIES = ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv']
NUM_CLASSES = len(CATEGORIES)
INPUT_SHAPE = (40, 112, 112, 3)  # (frames, height, width, channels)
BATCH_SIZE = 5
EPOCHS = 20

## Define function

In [8]:
def preprocess_data(category):
    X = []
    y = []
    exampleCount = 1
    category_dir = os.path.join(DATA2_DIR, category)
    print("Loading ", category)
        
    for video in os.listdir(category_dir):
        video_dir = os.path.join(category_dir, video)

        if len(os.listdir(video_dir)) > 0:
            print("Loading video ",str(exampleCount), ': ', str(video))  
            frames = []

            for frame_file in range(40):
                img_path = os.path.join(video_dir, str(str(frame_file) + '.jpg'))
                img = image.load_img(img_path, target_size=(112, 112))
                x = image.img_to_array(img)
                x = x/255
                x = np.expand_dims(x, axis=0)
                frames.append(x)
            X.append(np.concatenate(frames))
            y.append(category)
            exampleCount += 1
            frames.clear()
            gc.collect()

            for frame_file in range(40,80):
                img_path = os.path.join(video_dir, str(str(frame_file) + '.jpg'))
                img = image.load_img(img_path, target_size=(112, 112))
                x = image.img_to_array(img)
                x = x/255
                x = np.expand_dims(x, axis=0)
                frames.append(x)
            X.append(np.concatenate(frames))
            y.append(category)
            exampleCount += 1
            gc.collect()

        else:
            print('Skip vid: ', video)    
                
    print("Finished loading ", category)
    print("Formating input X")
    X = np.array(X)
    print("Formating label y")
    y = np.array(y)
    
    print('\nSummary: \n')
    print(str(category), ': ', str(exampleCount))
    
    return X, y

In [25]:
def extract_feature(X, model, step):
    start = 0
    end = 0
    while end < X.shape[0]:
        end = start + step
        if start != 0:
            print('Processing video ', start, ' - ', end)
            X_sliced = X[start:end,:,:,:,:]
            X_train = model.predict_on_batch(X_sliced)
            X_train_concat = np.concatenate((X_train_concat,X_train))
            start = end
            gc.collect()
        else:      
            print('Start processing...')
            print('Processing video ', start, ' - ', end)
            X_sliced = X[start:end,:,:,:,:]
            X_train = model.predict_on_batch(X_sliced)
            X_train_concat = X_train
            start = end
            gc.collect()

    print('Processed ', end, ' video')
    
    return X_train_concat

In [9]:
def load_processed_data(categories, X, model):
    if len(X) != 0:
        x_processed = np.load('/home/jeheng/Desktop/numpy/Validate_data_processed/'+ model + '/' + categories + '_processed.npy')
        X = np.concatenate((X, x_processed))
        return X, x_processed.shape
    else:
        X = np.load('/home/jeheng/Desktop/numpy/Validate_data_processed/'+ model + '/' + categories + '_processed.npy')
        return X, X.shape

In [10]:
def load_processed_label(categories, y, model):
    if len(y) != 0:
        y_processed = np.load('/home/jeheng/Desktop/numpy/Validate_data_processed/'+ model + '/' + categories + '_label.npy')
        y = np.concatenate((y, y_processed))
        return y, y_processed.shape
    else:
        y = np.load('/home/jeheng/Desktop/numpy/Validate_data_processed/'+ model + '/' + categories + '_label.npy')
        return y, y.shape

In [15]:
gc.collect()

154

In [ ]:
def train_test_load_data(categories, X_train, X_test):
    if len(X_train) != 0 and len(X_test) != 0:
        x_processed = np.load('f:/Kinetic_700/numpy2/' + categories + '_processed.npy')
        split = round(len(x_processed)*0.8)
        X_train = np.concatenate((X, x_processed[0:split]))
        X_test = np.concatenate((X, x_processed[split:len(x_processed)]))
        return X_train, X_test, x_processed.shape, split
    else:
        x_processed = np.load('f:/Kinetic_700/numpy2/' + categories + '_processed.npy')
        split = round(len(x_processed)*0.8)
        X_train = x_processed[0:split]
        X_test = x_processed[split:len(x_processed)]
        return X_train, X_test, x_processed.shape, split

In [ ]:
def train_test_load_label(categories, y_train, y_test):
    if len(y_train) != 0 and len(y_test) != 0:
        
        y_processed = np.load('f:/Kinetic_700/numpy2/' + categories + '_label.npy')
        split = round(len(y_processed)*0.8)
        
        y_train = np.concatenate((y_train, y_processed[0:split]))
        y_test = np.concatenate((y_train, y_processed[split:len(y_processed)]))
        
        return y_train, y_test, y_processed.shape, split
    else:
        y_processed = np.load('f:/Kinetic_700/numpy2/' + categories + '_label.npy')
        split = round(len(y_processed)*0.8)

        y_train = y_processed[0:split]
        y_test = y_processed[split:len(y_processed)]
        return y, y.shape

In [ ]:
def one_hot(y):
    y[y == 'dining'] = 0
    y[y == 'drawing'] = 1
    y[y == 'mopping_floor'] = 2
    y[y == 'reading_book'] = 3
    y[y == 'running_on_treadmill'] = 4
    y[y == 'sleeping'] = 5
    y[y == 'watching_tv'] = 6
    
    y_encode = to_categorical(y, num_classes=7)

    print("Shape of y_encode: ", y_encode.shape)
    
    return y_encode

## Define extractor model

In [13]:
# creating the base model of pre-trained VGG16 model
from keras.applications.vgg16 import VGG16

### ============================ VGG16 GlobalAVG  ============================

In [ ]:
# Load vgg16 model
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

# Freeze vgg16 layers
for layer in vgg16.layers:
    layer.trainable = False

# Extract features using vgg16
inputs = Input(shape=INPUT_SHAPE)
l1 = TimeDistributed(vgg16)(inputs)
l2 = TimeDistributed(GlobalAveragePooling2D())(l1)
#outputs = TimeDistributed(Flatten())(l2)

# Create model
vgg16_gavg = Model(inputs, l2)
vgg16_gavg.summary()


### ============================ VGG16 AVG ============================

In [15]:
# Load vgg16 model
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

# Freeze vgg16 layers
for layer in vgg16.layers:
    layer.trainable = False

# Extract features using vgg16
inputs = Input(shape=INPUT_SHAPE)
l1 = TimeDistributed(vgg16)(inputs)
l2 = TimeDistributed(AveragePooling2D(pool_size=(2,2),strides=1))(l1)
outputs = TimeDistributed(Flatten())(l2)

# Create model
vgg16_avg = Model(inputs, outputs)
vgg16_avg.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 40, 112, 112, 3)  0         
                             ]                                   
                                                                 
 time_distributed_3 (TimeDis  (None, 40, 3, 3, 512)    14714688  
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 40, 2, 2, 512)    0         
 tributed)                                                       
                                                                 
 time_distributed_5 (TimeDis  (None, 40, 2048)         0         
 tributed)                                                       
                                                                 
Total params: 14,714,688
Trainable params: 0
Non-trainable 

### ============================ Resnet GlobalAVG ============================

In [32]:
# Load ResNet50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

# Freeze ResNet50 layers
for layer in resnet.layers:
    layer.trainable = False

# Extract features using ResNet50
inputs = Input(shape=INPUT_SHAPE)
l1 = TimeDistributed(resnet)(inputs)
l2 = TimeDistributed(GlobalAveragePooling2D())(l1)
#outputs = TimeDistributed(Flatten())(l2)

# Create model
resnet_gavg = Model(inputs, l2)
resnet_gavg.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 40, 112, 112, 3)  0         
                             ]                                   
                                                                 
 time_distributed_6 (TimeDis  (None, 40, 4, 4, 2048)   23587712  
 tributed)                                                       
                                                                 
 time_distributed_7 (TimeDis  (None, 40, 2048)         0         
 tributed)                                                       
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


## Process

In [ ]:
#class -- ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv']

In [241]:
category = 'watching_tv'

In [242]:
X, y = preprocess_data(category)
gc.collect()

Loading  watching_tv
Loading video  1 :  illbIdQ0W0g_000002_000012
Loading video  3 :  iMLgP40MaMs_000017_000027
Loading video  5 :  ImvV7a00Iso_000036_000046
Loading video  7 :  ItfZ6b8nduo_000012_000022
Loading video  9 :  IX864cuGCXg_000008_000018
Loading video  11 :  i_CNmgtgpvg_000033_000043
Loading video  13 :  I_QxQbSyPGA_000040_000050
Loading video  15 :  j8ilCoKhaxU_000003_000013
Loading video  17 :  JHZrmSX7Sko_000005_000015
Loading video  19 :  jlDKyHzuwd4_000123_000133
Loading video  21 :  JYBOtrMTahc_000039_000049
Loading video  23 :  Jz5qdEgpmA4_000005_000015
Loading video  25 :  JzOx6KiV1f8_000000_000010
Loading video  27 :  k659P60d5TM_000013_000023
Loading video  29 :  Kh6wv2EMqbo_000000_000010
Loading video  31 :  KRsGr1Kq9ak_000063_000073
Loading video  33 :  KRzV6wDS000_000002_000012
Loading video  35 :  K_QuvGRl7PQ_000000_000010
Loading video  37 :  lhb4IAWSxKE_000027_000037
Loading video  39 :  L_dzgn4x5KY_000010_000020
Loading video  41 :  mbD5loJ8FH8_000069_0000

0

In [243]:
print("Shape of X: ", X.shape)
print("Shape of y: ", y.shape)

Shape of X:  (98, 40, 112, 112, 3)
Shape of y:  (98,)


In [244]:
sys.getsizeof(X)

590069928

In [245]:
gc.collect()

0

In [246]:
np.save(category + '_full_validate', X)
np.save(category + '_full_label_validate', y)

## Extract Feature

### VGG16_AVG

In [247]:
model_name = 'vgg16_avg'
X_processed = extract_feature(X, vgg16_avg, 4)
gc.collect()

Start processing...
Processing video  0  -  4
Processing video  4  -  8
Processing video  8  -  12
Processing video  12  -  16
Processing video  16  -  20
Processing video  20  -  24
Processing video  24  -  28
Processing video  28  -  32
Processing video  32  -  36
Processing video  36  -  40
Processing video  40  -  44
Processing video  44  -  48
Processing video  48  -  52
Processing video  52  -  56
Processing video  56  -  60
Processing video  60  -  64
Processing video  64  -  68
Processing video  68  -  72
Processing video  72  -  76
Processing video  76  -  80
Processing video  80  -  84
Processing video  84  -  88
Processing video  88  -  92
Processing video  92  -  96
Processing video  96  -  100
Processed  100  video


0

### Resnet_GAVG

In [251]:
model_name = 'resnet_gavg'
X_processed = extract_feature(X, resnet_gavg, 4)
gc.collect()

Start processing...
Processing video  0  -  4
Processing video  4  -  8
Processing video  8  -  12
Processing video  12  -  16
Processing video  16  -  20
Processing video  20  -  24
Processing video  24  -  28
Processing video  28  -  32
Processing video  32  -  36
Processing video  36  -  40
Processing video  40  -  44
Processing video  44  -  48
Processing video  48  -  52
Processing video  52  -  56
Processing video  56  -  60
Processing video  60  -  64
Processing video  64  -  68
Processing video  68  -  72
Processing video  72  -  76
Processing video  76  -  80
Processing video  80  -  84
Processing video  84  -  88
Processing video  88  -  92
Processing video  92  -  96
Processing video  96  -  100
Processed  100  video


0

### VGG16_GAVG

In [ ]:
model_name = 'vgg16_gavg'
X_processed = extract_feature(X, resnet_gavg, 4)
gc.collect()

In [252]:
print('Final shape of X: ', X_processed.shape)
print('Size of X: ', sys.getsizeof(X_processed))

Final shape of X:  (98, 40, 2048)
Size of X:  32112776


In [253]:
np.save('F:/Kinetic_700/'+ model_name + '/' + category + '_processed', X_processed)
np.save('F:/Kinetic_700/'+ model_name + '/' + category + '_label', y)

print('Processed data saved to disk')

Processed data saved to disk


In [254]:
X_reload = np.load('F:/Kinetic_700/'+ model_name + '/' + category + '_processed.npy')
y_reload = np.load('F:/Kinetic_700/'+ model_name + '/' + category + '_label.npy')

print(X_reload.shape)
print(y_reload.shape)

(98, 40, 2048)
(98,)


### Free Ram

In [240]:
del X_processed, X, y, X_reload, y_reload
gc.collect()

22

## Split Train Validation

In [280]:
gc.collect()

1547

In [267]:
def train_test_loader(classname, X_train, X_test, y_train, y_test, model_name):

    if len(X_train) != 0 and len(y_train) != 0:
        x_processed = np.load('f:/Kinetic_700/Train_data_processed/' + model_name + '/' + classname + '_processed.npy')
        split = round(len(x_processed)*0.8)
        X_train = np.concatenate((X_train, x_processed[0:split, :, :]))
        X_test = np.concatenate((X_test, x_processed[split:len(x_processed), :, :]))

        y_processed = np.load('f:/Kinetic_700/Train_data_processed/' + model_name + '/' + classname + '_label.npy')     
        y_train = np.concatenate((y_train, y_processed[0:split]))
        y_test = np.concatenate((y_test, y_processed[split:len(y_processed)]))
        
        return X_train, X_test, y_train, y_test, x_processed.shape, y_processed.shape, split
    else:
        x_processed = np.load('f:/Kinetic_700/Train_data_processed/' + model_name + '/' + classname + '_processed.npy')
        split = round(len(x_processed)*0.8)
        X_train = x_processed[0:split, :, :]
        X_test = x_processed[split:len(x_processed), :, :]
        
        y_processed = np.load('f:/Kinetic_700/Train_data_processed/' + model_name + '/' + classname + '_label.npy')
        y_train = y_processed[0:split]
        y_test = y_processed[split:len(y_processed)]

        
        return X_train, X_test, y_train, y_test, x_processed.shape, y_processed.shape, split

In [257]:
categories = ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv']

In [281]:
X_train = [] 
X_test = []
y_train = []
y_test = []

for classname in categories:
    X_train, X_test, y_train, y_test, current_data_shape, current_label_shape, split = train_test_loader(classname, X_train, X_test, y_train, y_test, 'resnet_gavg')
    
    print("Shape of ", classname, " data set: ", current_data_shape)
    print("Shape of ", classname, " label set: ", current_label_shape)
    print("Shape of X_train: ", X_train.shape)
    print("Shape of y_train: ", y_train.shape)
    print("Shape of X_test: ", X_test.shape)
    print("Shape of y_test: ", y_test.shape, '\n')


Shape of  dining  data set:  (1500, 40, 2048)
Shape of  dining  label set:  (1500,)
Shape of X_train:  (1200, 40, 2048)
Shape of y_train:  (1200,)
Shape of X_test:  (300, 40, 2048)
Shape of y_test:  (300,) 

Shape of  drawing  data set:  (1476, 40, 2048)
Shape of  drawing  label set:  (1476,)
Shape of X_train:  (2381, 40, 2048)
Shape of y_train:  (2381,)
Shape of X_test:  (595, 40, 2048)
Shape of y_test:  (595,) 

Shape of  mopping_floor  data set:  (1688, 40, 2048)
Shape of  mopping_floor  label set:  (1688,)
Shape of X_train:  (3731, 40, 2048)
Shape of y_train:  (3731,)
Shape of X_test:  (933, 40, 2048)
Shape of y_test:  (933,) 

Shape of  reading_book  data set:  (1872, 40, 2048)
Shape of  reading_book  label set:  (1872,)
Shape of X_train:  (5229, 40, 2048)
Shape of y_train:  (5229,)
Shape of X_test:  (1307, 40, 2048)
Shape of y_test:  (1307,) 

Shape of  running_on_treadmill  data set:  (1630, 40, 2048)
Shape of  running_on_treadmill  label set:  (1630,)
Shape of X_train:  (6533, 

In [282]:
y_train[y_train == 'dining'] = 0
y_train[y_train == 'drawing'] = 1
y_train[y_train == 'mopping_floor'] = 2
y_train[y_train == 'reading_book'] = 3
y_train[y_train == 'running_on_treadmill'] = 4
y_train[y_train == 'sleeping'] = 5
y_train[y_train == 'watching_tv'] = 6

y_test[y_test == 'dining'] = 0
y_test[y_test == 'drawing'] = 1
y_test[y_test == 'mopping_floor'] = 2
y_test[y_test == 'reading_book'] = 3
y_test[y_test == 'running_on_treadmill'] = 4
y_test[y_test == 'sleeping'] = 5
y_test[y_test == 'watching_tv'] = 6

In [283]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

print("Shape of y_train: ", y_train.shape)
print("Shape of y_test: ", y_test.shape)

Shape of y_train:  (8769, 7)
Shape of y_test:  (2191, 7)


In [284]:
np.save('X_train', X_train)
np.save('y_train', y_train)
np.save('X_test', X_test)
np.save('y_test', y_test)

### To load manually

In [ ]:
y = []

for classname in CATEGORIES:
    y, shape = load_processed_label(classname, y)
    
    print("Shape of ", classname, " set: ", shape)
    print("Shape of y: ", y.shape)

In [ ]:
y[y == 'dining'] = 0
y[y == 'drawing'] = 1
y[y == 'mopping_floor'] = 2
y[y == 'reading_book'] = 3
y[y == 'running_on_treadmill'] = 4
y[y == 'sleeping'] = 5
y[y == 'watching_tv'] = 6

In [ ]:
y_encoded = to_categorical(y, num_classes=7)
print("Shape of y_encoded: ", y_encoded.shape)

In [ ]:
X = []

for classname in CATEGORIES:
    X, shape = load_processed_data(classname, X)
    
    print("Shape of X: ", X.shape)
    print("Shape of ", classname , ': ', shape)


In [ ]:
print("Shape of X: ", X.shape)
print("Shape of y: ", y_encoded.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test: ", y_test.shape)

In [ ]:
np.save('X_train', X_train)
np.save('y_train', y_train)
np.save('X_test', X_test)
np.save('y_test', y_test)

### Load validation set

In [ ]:
X_eval = []
y_eval = []

for classname in CATEGORIES:
    X_eval, X_current_shape = load_processed_data(classname,X_eval, 'resnet_gavg')
    y_eval, y_current_shape = load_processed_label(classname,y_eval, 'resnet_gavg')
    

    print("Shape of ", classname, " data set: ", X_current_shape)
    print("Shape of ", classname, " label set: ", y_current_shape)
    print("Shape of X_eval: ", X_eval.shape)
    print("Shape of y_eval: ", y_eval.shape, '\n')

In [ ]:
np.save('X_eval', X_eval)
np.save('y_eval', y_eval)